In [88]:
import pandas as pd
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import Reader

In [89]:
def getprediction( predictions, userId ):
    user_pred = []
    for uid, iid, true_r, est, _ in predictions:
        if uid == userId :
            user_pred.append( (iid, est) )
    df_userp = pd.DataFrame( user_pred, columns = [ 'movieId', 'prediction'] )
    df_userp['movieId'] = df_userp['movieId'].astype(int)
    return( df_userp )

In [90]:
#load data
df_small = pd.read_csv('../movielens_small/ratings_small.csv')
df_mov = pd.read_csv( '../movielens_small/movies.csv' )
#df_small.head()

In [91]:
reader = Reader()
rating_data = Dataset.load_from_df(df_small[['userId', 'movieId', 'rating']], reader)
#rating_data.split(n_folds=5)

In [92]:
algo = SVD( biased = False, n_factors = 60, n_epochs = 50 )
trainset = rating_data.build_full_trainset()
algo.fit(trainset)

In [93]:
# Then predict ratings for all pairs (u, i) that are not in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [94]:
user_id = 668

In [95]:
df_userp = getprediction( predictions, user_id )
#df_userp.head()
df_userp = ( df_userp. join(df_mov.set_index('movieId'), on = 'movieId' ).
            sort_values('prediction', ascending=False) )
df_userp.head(15)

,movieId,prediction,title,genres
153,858,4.885717,"Godfather, The (1972)",Crime|Drama
117,2858,4.744878,American Beauty (1999),Drama|Romance
24,50,4.716306,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
78,527,4.696700,Schindler's List (1993),Drama|War
2191,905,4.642328,It Happened One Night (1934),Comedy|Romance
381,1276,4.631842,Cool Hand Luke (1967),Drama
2137,898,4.601359,"Philadelphia Story, The (1940)",Comedy|Drama|Romance
1021,3462,4.542967,Modern Times (1936),Comedy|Drama|Romance
376,111,4.515132,Taxi Driver (1976),Crime|Drama|Thriller
118,2959,4.507248,Fight Club (1999),Action|Crime|Drama|Thriller


In [96]:
df_user = ( df_small.loc[ df_small['userId'] == user_id ].
           join(df_mov.set_index('movieId'),on = 'movieId').
           reset_index(drop = True).
           sort_values('rating', ascending=False) )
df_user

,userId,movieId,rating,timestamp,title,genres
0,668,296,5.0,993613478,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,668,593,5.0,993613478,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
3,668,608,5.0,993613478,Fargo (1996),Comedy|Crime|Drama|Thriller
14,668,2997,5.0,993613478,Being John Malkovich (1999),Comedy|Drama|Fantasy
13,668,2908,5.0,993613415,Boys Don't Cry (1999),Drama
6,668,1213,5.0,993613359,Goodfellas (1990),Crime|Drama
7,668,1221,5.0,993613196,"Godfather: Part II, The (1974)",Crime|Drama
11,668,2324,5.0,993613359,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War
1,668,318,4.0,993613478,"Shawshank Redemption, The (1994)",Crime|Drama
9,668,1358,4.0,993613415,Sling Blade (1996),Drama
